In [1]:
import pandas as pd
df = pd.read_csv('./data/nyc_taxi.csv')

In [5]:
### EXAMPLE WITH A COORDINATE
import geopandas as gpd
from shapely.geometry import Point

barrios = gpd.read_file("./data/barrios_nyc.geojson")

origen = Point(-73.985428, 40.748817)

punto = barrios[barrios.contains(origen)]
print(punto['boro_name'])

63    Manhattan
Name: boro_name, dtype: object


In [6]:
### APPLY TO DATASET
gdf_origin = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['pickup_longitude'], df['pickup_latitude']),
    crs="EPSG:4326" 
)

gdf_origin = gdf_origin.sjoin(barrios, how="left", predicate="intersects")
gdf_origin = gdf_origin.rename(columns={"boro_name": "neigh_origin"})

df['neigh_origin'] = gdf_origin['neigh_origin']

gdf_destination = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['dropoff_longitude'], df['dropoff_latitude']),
    crs="EPSG:4326"
)

gdf_destination = gdf_destination.sjoin(barrios, how="left", predicate="intersects")
gdf_destination = gdf_destination.rename(columns={"boro_name": "neigh_destination"})

df['neigh_destination'] = gdf_destination['neigh_destination']


In [7]:
df[['neigh_origin','neigh_destination']]

,neigh_origin,neigh_destination
0,Brooklyn,Brooklyn
1,Manhattan,Manhattan
2,Manhattan,Manhattan
3,Queens,Manhattan
4,Manhattan,Brooklyn
...,...,...
10994,Manhattan,Manhattan
10995,Manhattan,Manhattan
10996,Manhattan,Manhattan
10997,Manhattan,Manhattan


In [8]:
df.to_csv('./data/nyc_taxi_neigh.csv') #export as nyc_taxi_neigh.csv